## **FE5213 Project**


This project accounts for 50% of your overall grade. You are encouraged to work in groups of $3-5$. Please submit only 1 copy of your project work per group to your TA's email address (zhongxi.zheng@u.nus.edu) by the submission deadline.

This project contains two parts (Parts A and B). Undergraduate students are allowed to choose either Part A or Part B. Graduate students must choose Part B. In particular, a group should work on Part B of the project if it contains at least 1 graduate student. Each group is  welcome (but not required) to work on both parts of the project. Extra credits can be earned by doing that.

You are encouraged to work closely with your groupmates as well as across groups. However, we will not assist you with these projects.



**Deadline**: Monday April 22 (23:59)

---

#### **Part A**

Please prepare a Jupyter notebook that provides answers to all parts of all questions asked in quiz 2. The question sheet is uploaded to Canvas. 


#### **Part B**

This part of the project is to prepare a Jupyter notebook that constructs a Python class that allows a user to  construct instances of a random life-time income sequence that emerge after graduating from a professional school that requires $k$ years of training for a person who then works for $T+1-k$ years.

The class should assume that professional income is governed by a linear 
state space system like the one described in this 
quantecon lecture:

  <https://python.quantecon.org/linear_models.html>
  
 Note that some  properties of  a  linear state system can be  analyzed by using the quantecon **LinearStateSpace** class.
 
(We anticipate that the new class that you will create will want to call methods from the  **LinearStateSpace** class.)


In your class, please make  the present value of entering school for  a profession   that requires $k$ years of schooling at time $t = 0$ be the random variable

$$
PV_0 = \sum_{t=k}^{T} \beta^t y_{t}
$$ 

where 

* an annual income process $\{y_{t+j}\}_{j=0}^\infty$ is governed by an instance of a linear state space system described in equation (26.1) of the quantecon lecture on linear state space systems
<https://python.quantecon.org/linear_models.html>
* $\beta \in (0,1)$ is a scalar discount factor

* $A, C, G, \mu_0, \Sigma_0$ pin down the income process and the Markov state vector $x_t \in \mathbb{R}^n$
associated with  it.

* $k\geq0$ is the number of years of schooling required. When $k=0$, this corresponds an instance where the agent does **not** go to a professional school, which means working during times $t=0,1,\dots, T$.
When $k =1$, this corresponds to an instance in which the agent goes to school for one year and that works 
during years $t=1,\dots, T$. And so on.


A  two-parameter **person** has  mean-variance utility 

$$
U(\mu,\sigma^2) = a \mu - \frac{b}{2} \sigma^2
$$

where $a$ and $b$ are two positive scalars that describe the person's attitude toward bearing risk associated with a random variable with mean $\mu$ and variance $\sigma^2$. 






## Features of the Python class 

Name the Python class as you wish.

Possible names might be "career" or "profession" or $\ldots$.

As **inputs** to creating an instance, your class should include

* matrices  $A, C, G, \mu_0, \Sigma_0$ that set up an income process

* a discount factor $\beta \in (0,1)$

* $T$ - an integer that determines maximum career length

* $k$ - an integer in $[0, 1, \ldots, T-1]$ that describes years of schooling required to enter a profession

* a test for whether $k \in [0, 1, \ldots, T-1]$. The test should gently warn  a user when the user has tried to specify  an impossible situation 

* scalar parameters $a$ and $b$ that describe a **person**'s attitudes toward risk


## Methods associated with the class

* a method that takes as inputs $A, C, G, \mu_0, \Sigma_0$ and outputs an **income process**

* a method that takes parameters $T, k$ and an income process as inputs and outputs a **profession**
(you could also call it a "professional school" if you want)


* a method that  takes a profession as an input and computes the unconditional mean $ \mathbb{E} PV_0$  of entering school for the profession at time $0$

* a method that computes takes a profession as an input the unconditional variance of $ \mathbb{E}( PV_0 - \mathbb{E} PV_0)^2$ of entering school for the profession at time $0$.

* a method that takes a profession as an input  and  computes the conditional mean $ \mathbb{E} [PV_0 | x_0]$ of entering school for a profession at time $0$, conditional on a realization of $x_0$ at time $0$.

* a method that  takes a profession as an input and computes the conditional variance $ \mathbb{E}[(PV_0 - \mathbb{E} [PV_0 | x_0])^2 | x_0] $ of entering school for a profession at time $0$, condional on a realization  $x_0$ at  time $0$.

* a method that takes $a$ and $b$ as inputs and outputs a   **person** with a mean-variance utility function

* a method that takes a **person** and a **profession** as inputs and computes an unconditional expectation of expected utility associated with entering that professional school at time $0$.


* a method that takes a **person** and a **profession** as inputs and computes  expected utility associated with entering that professional school at time $0$, conditional on a realization of the state $x_0$ at time $0$.



## Examples

Having created the Python class, please create instance  of **three** different professions and **two** different persons and tell how those different persons might choose to enter different professions.

For this part, try to have some fun by creating examples that show the power of your Python class.

## Project Evaluation Methods

After you submit your project, we shall write our own Jupyter notebook and stress-test your class by using it to analyze instances of professions and persons that we shall construct. 

In [90]:
from warnings import warn
import quantecon as qe
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple

class Career:
    
    def __init__(self,A,C,G,mu0,sigma0,beta,T,k,a,b):
        #test for valid k and warn user
        if k < 0 or k > T-1:
            warn("k value is invalid! k value should be bounded between 0 and T - 1!")
        self.A = A
        self.C = C
        self.G = G
        self.mu0 = mu0
        self.sigma0 = sigma0
        self.beta = beta
        self.T = T
        self.k = k
        self.a = a
        self.b = b
    
    def create_income_process(self,A,C,G,mu0,sigma0):
        process = qe.LinearStateSpace(A,C,G,mu_0 = mu0, Sigma_0 = sigma0)
        
        return process
    
    def create_profession(self,T,k,income_process):
        Profession = namedtuple('Profession',['T','k','income_process','beta'])
        profession = Profession(T,k,income_process,self.beta)
        
        return profession
        
    def get_EPV0(self, profession):
        #get y_t
        mm0 = profession.income_process.moment_sequence()
        mu_y = []

        for _ in range(T):
            m_x, m_y, S_x, S_y = next(mm0)
            mu_y.append(float(m_y))
        
        #get y_t from k to T-1
        mu_y_k = mu_y[k:]
        
        #create discount factor matrix from k to T-1
        ex = np.arange(k,T)
        beta_m = np.power(profession.beta,ex)
        
        return beta_m @ mu_y_k
    
    def get_var0(self, profession):
        #get Sigma_y
        mm0 = profession.income_process.moment_sequence()
        Sigma_y = []

        for _ in range(T):
            m_x, m_y, S_x, S_y = next(mm0)
            Sigma_y.append(float(S_y))
        
        #get Sigma_y from k to T-1
        Sigma_y_k = Sigma_y[k:]
        
        #create discount factor matrix from k to T-1
        ex = np.arange(k,T)
        beta_m = np.power(profession.beta,ex)
        
        return beta_m @ Sigma_y_k
    
    def get_con_EP0(self, profession):
        # use formula given in lecture notes (L6)
        
        #how to make sure x0 here is same as x0 when you simulate it
        
        G = profession.income_process.G
        A = profession.income_process.A
        k = profession.k
        T = profession.T
        size = A.shape[0]
        bA = profession.beta * A
        H = G @ (np.linalg.matrix_power(bA, k) 
                  @ (np.linalg.inv(np.eye(size) - bA) 
                      @ np.linalg.inv(np.eye(size) - np.linalg.matrix_power(bA, T + 1 - k)
                                     )
                    )
                )
        
        x, y = profession.income_process.simulate(1)
        x0 = x
        return H @ x0
    
    def get_con_var0(self, profession):
        G = profession.income_process.G
        A = profession.income_process.A
        C = profession.income_process.C3
        
        #find covariance matrix j-step ahead for x
        
    
    def create_person(self,a,b):
        def util(mu,var):
            return a*mu - (b/2)*var
        return util
    
    
        